## Raw to csv preprocessing

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import random
from utility.preprocessing3 import clean_data, preprocessing, smote,normalise_data, preprocess_column, preprocess_column,split_data
from models.model3 import train_model
from testing.evaluation3 import evaluation_data,infer
DATA_SIZE=10000
BASE_PATH=f'data/{DATA_SIZE}'

In [15]:
clean_data(BASE_PATH)

process start..


  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\pedne\Desktop\onkar\Upwork\PyTOrch_LSTM\utility\preprocessing3.py:39: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  _df['new']=p
100%|██████████| 100/100 [00:02<00:00, 41.72it/s]


preprocessing done


### Feature engineering 

In [8]:
df=pd.read_csv(f"{BASE_PATH}/process/final_{DATA_SIZE}.csv")
df.dtypes

PatientID                                   object
PatientGender                               object
PatientDateOfBirth                          object
PatientRace                                 object
PatientMaritalStatus                        object
PatientLanguage                             object
PatientPopulationPercentageBelowPoverty    float64
AdmissionID                                  int64
AdmissionStartDate                          object
target_days                                  int64
days                                         int64
PrimaryDiagnosisCode                        object
PrimaryDiagnosisDescription                 object
dtype: object

In [9]:
# check data len 
#DAYS=min(df['days']) 2971
DAYS=60
print(f"total Admission : {len(df)}")
print(f"admit within {DAYS} : {len(df[df['target_days']<=DAYS])} ")


total Admission : 372
admit within 60 : 4 


In [10]:
all_feature=['PatientID','PatientGender', 'PatientRace','Age',
'PatientMaritalStatus',
'PatientPopulationPercentageBelowPoverty', 'AdmissionID','days',
'PrimaryDiagnosisCode','Target']

### Process_df and train_df we can use for evaluation

In [11]:
#convert df in lower case
#df=lower_case(df)
process_df=preprocessing(df,DAYS,all_feature) # whole dataset you will for evaluation
major_count=process_df['Target'].value_counts()[0]
train_df=smote(process_df) #upsample low class
train_df=normalise_data(train_df,BASE_PATH)
train_df.to_csv(f'{BASE_PATH}/train.csv',index=False)
#train_df

In [12]:
train_df['Target'].value_counts()

0    368
1    368
Name: Target, dtype: int64

### Train model and save

In [3]:
train_df=pd.read_csv(f'{BASE_PATH}/train.csv')
train_df

,PatientID,PatientGender,PatientRace,Age,PatientMaritalStatus,PatientPopulationPercentageBelowPoverty,AdmissionID,days,PrimaryDiagnosisCode,Target
0,965,1,1,28,2,13.710000,2,7,1188,0
1,6326,0,1,66,3,14.770000,8,11,2463,0
2,134,1,1,32,1,17.100000,2,11,2052,0
3,7883,1,1,47,3,15.400000,2,17,537,0
4,1834,1,1,18,1,1.330000,1,19,2044,0
...,...,...,...,...,...,...,...,...,...,...
72019,5339,1,3,55,3,13.053076,6,4,1802,1
72020,1515,1,0,51,0,13.141664,6,12,2373,1
72021,546,0,3,23,1,14.940520,2,15,483,1
72022,1128,0,3,56,3,95.711892,5,16,2255,1


In [4]:
X_train,y_train,X_test,y_test,X_val,y_val=split_data(train_df)
X_val.shape

(14044, 9, 1)

In [5]:
history = train_model(X_train,y_train,X_val,y_val,DATA_SIZE)

Epoch 1/50
1801/1801 [==============================] - 27s 13ms/step - loss: 0.2194 - accuracy: 0.6461 - val_loss: 0.2012 - val_accuracy: 0.6894

Epoch 00001: val_accuracy improved from -inf to 0.68940, saving model to save_model\model_10000\model.h5
Epoch 2/50
1801/1801 [==============================] - 21s 12ms/step - loss: 0.1899 - accuracy: 0.7146 - val_loss: 0.1698 - val_accuracy: 0.7621

Epoch 00002: val_accuracy improved from 0.68940 to 0.76210, saving model to save_model\model_10000\model.h5
Epoch 3/50
1801/1801 [==============================] - 22s 12ms/step - loss: 0.1671 - accuracy: 0.7621 - val_loss: 0.1621 - val_accuracy: 0.7716

Epoch 00003: val_accuracy improved from 0.76210 to 0.77165, saving model to save_model\model_10000\model.h5
Epoch 4/50
1801/1801 [==============================] - 22s 12ms/step - loss: 0.1602 - accuracy: 0.7738 - val_loss: 0.1546 - val_accuracy: 0.7810

Epoch 00004: val_accuracy improved from 0.77165 to 0.78097, saving model to save_model\mode

1801/1801 [==============================] - 8s 4ms/step - loss: 0.1427 - accuracy: 0.7994 - val_loss: 0.1396 - val_accuracy: 0.8064

Epoch 00037: val_accuracy did not improve from 0.80639
Epoch 38/50
1801/1801 [==============================] - 7s 4ms/step - loss: 0.1427 - accuracy: 0.8002 - val_loss: 0.1396 - val_accuracy: 0.8064

Epoch 00038: val_accuracy did not improve from 0.80639
Epoch 39/50
1801/1801 [==============================] - 7s 4ms/step - loss: 0.1426 - accuracy: 0.7998 - val_loss: 0.1396 - val_accuracy: 0.8064

Epoch 00039: val_accuracy did not improve from 0.80639
Epoch 40/50
1801/1801 [==============================] - 8s 4ms/step - loss: 0.1427 - accuracy: 0.7997 - val_loss: 0.1396 - val_accuracy: 0.8064

Epoch 00040: val_accuracy did not improve from 0.80639
Epoch 41/50
1801/1801 [==============================] - 8s 4ms/step - loss: 0.1426 - accuracy: 0.8007 - val_loss: 0.1396 - val_accuracy: 0.8064

Epoch 00041: val_accuracy did not improve from 0.80639
Epoch 4

### Evaluation 

In [7]:
try:
    X_test
except NameError:
    X_train,y_train,X_test,y_test,X_val,y_val=split_data(train_df)
    
df_pred=evaluation_data(X_test,y_test,DATA_SIZE,False)
df_pred

12/12 [==============================] - 0s 2ms/step - loss: 0.1427 - accuracy: 0.8061
Evaluation result:  [0.14266926050186157, 0.8060941696166992]


,Actual,Predictions
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
356,0,1
357,1,1
358,0,0
359,1,1


In [8]:
positives=len(df_pred[df_pred['Actual'] == df_pred['Predictions']])
negatives=len(df_pred[df_pred['Actual'] != df_pred['Predictions']])
print('No of correct predictions are {} and false predictions are {} on test dataset'.format(positives,negatives))

No of correct predictions are 291 and false predictions are 70 on test dataset


In [12]:
# df_test=evaluation_data(train_df,'save_model/model.pkl')
print(f"total={len(df_pred)}\n")
print(df_pred.Actual.value_counts())
print(f"\n0 <-> 0 = {len(df_pred[(df_pred['Actual']==0) & (df_pred['Predictions']==0)])}")
print(f"1 <-> 1 = {len(df_pred[(df_pred['Actual']==1) & (df_pred['Predictions']==1)])}")
print(f"0 <-> 1 = {len(df_pred[(df_pred['Actual']==0) & (df_pred['Predictions']==1)])}")
print(f"1 <-> 0 = {len(df_pred[(df_pred['Actual']==1) & (df_pred['Predictions']==0)])}")

total=361

0    190
1    171
Name: Actual, dtype: int64

0 <-> 0 = 129
1 <-> 1 = 162
0 <-> 1 = 61
1 <-> 0 = 9


### inferences 

In [15]:
import pandas as pd
data={'PatientID':['886B5885-1EE2-49F3-98D5-A2F02EB8A9D4','0a9ba3e4-cf3c-49c4-9774-5eea2ee7d123'],
      'PatientGender':['Female','Male'],
      'PatientRace':['African American','White'],
      'Age':[21,32],
      'PatientMaritalStatus':['Single','Married'],
      'PatientPopulationPercentageBelowPoverty':[20.53,14.3],
      'AdmissionID':[2,3],
      'PrimaryDiagnosisCode':['C79.02','T70.3'],
     }
test_df=pd.DataFrame(data)
test_copy=test_df.copy()
test_df

,PatientID,PatientGender,PatientRace,Age,PatientMaritalStatus,PatientPopulationPercentageBelowPoverty,AdmissionID,PrimaryDiagnosisCode
0,886B5885-1EE2-49F3-98D5-A2F02EB8A9D4,Female,African American,21,Single,20.53,2,C79.02
1,0a9ba3e4-cf3c-49c4-9774-5eea2ee7d123,Male,White,32,Married,14.30,3,T70.3


In [13]:
main_df=pd.read_csv(f'{BASE_PATH}/AdmissionsCorePopulatedTable.txt', 
                              names=['PatientID','AdmissionID','AdmissionStartDate','AdmissionEndDate'], skiprows=1, sep='\t')
mappings=pd.read_csv(f'{BASE_PATH}/mappings.csv')
pcode=[]
dcode=[]
days=[]
for index, row in test_df.iterrows():
    if row['PatientID'] in mappings.PatientID.values:
        val=mappings[mappings['PatientID'] == row['PatientID']]['Patient_Code'].values[0]
        pcode.append(val)
    else:
        val=max(mappings.Patient_Code.values)+1
        pcode.append(val)
        
    if row['PrimaryDiagnosisCode'] in mappings.PrimaryDiagnosisCode.values:
        val=mappings[mappings['PrimaryDiagnosisCode'] == row['PrimaryDiagnosisCode']]['Diag_Code'].values[0]
        dcode.append(val)
    else:
        val=random.choice(mappings.Diag_Code.values.tolist())
        dcode.append(val)
        
    day=random.randint(1,20)
    days.append(day)
    
test_df['days']=days
test_df['PatientID']=pcode
test_df['PrimaryDiagnosisCode']=dcode
test_df=preprocess_column(test_df,BASE_PATH)

        
        
    

In [14]:
preds=infer(test_df,DATA_SIZE)
test_copy['Predictions']=preds
test_copy

,PatientID,PatientGender,PatientRace,Age,PatientMaritalStatus,PatientPopulationPercentageBelowPoverty,AdmissionID,PrimaryDiagnosisCode,Predictions
0,886B5885-1EE2-49F3-98D5-A2F02EB8A9D4,Female,African American,21,Single,20.53,2,C79.02,0
1,0a9ba3e4-cf3c-49c4-9774-5eea2ee7d123,Male,White,32,Married,14.30,3,T70.3,0
